In [1]:
import numpy as np
import pandas as pd
from imageio import imread
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
import os
from common import *
from models import own_LinRegGD
#import cv2
#import tensorflow as tf
from models import NeuralNetwork, Layer, Fixed_layer, binary_classifier, relu

print(f"Root directory: {os.getcwd()}")
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
    "font.size": 10,
})

#%matplotlib

Root directory: c:\Users\andre\Dropbox\FYS-STK4155_projects\FYS-STK4155 - Project2\FYS-STK4155-Prj2_report\code


# XOR classification
The following code checks that a Fixed neural net is able to correctly classify the XOR problem

In [2]:
np.random.seed(4155)
X = np.array([[1, 0],
              [0, 1],
              [1, 1],
              [0, 0]])

hidden1 = Fixed_layer(nbf_inputs=2, nbf_outputs=2, weights=np.array([[1,1],[1,1]]), bias=np.array([-1.5, -0.5]), activation_function=binary_classifier)
output = Fixed_layer(nbf_inputs=2, nbf_outputs=1, weights=np.array([-1,1]), bias=np.array([-0.5]), activation_function=binary_classifier)

model = NeuralNetwork()
model.add_layer(hidden1)
model.add_layer(output)

y_hat = model.predict(X)
print(f"y_hat: {y_hat}")

y_hat: [1 1 0 0]


In [3]:
np.random.seed(4155)
X = np.array([[1., 0.],
              [1., 1.],
              
              [0., 1.]])

X_test = np.array([[0., 0.]])

t = np.array([[1.,0.,1.]]).reshape(-1,1)

#hidden1 = Layer(nbf_inputs=2, nbf_outputs=4, activation_function=relu)
#hidden2 = Layer(nbf_inputs=hidden1.output, nbf_outputs=16)
hidden1 = Layer(nbf_inputs=2, nbf_outputs=2, activation_function=relu, name="hidden1")
output = Layer(nbf_inputs=hidden1.output, nbf_outputs=1, name="output")

model2 = NeuralNetwork()
model2.add_layer(hidden1)
model2.add_layer(output)

epochs = 100
batch_size = 1
model2.train_model(X, t, batch_size=batch_size, epochs=epochs)
y_hat = model2.predict(X_test)
print(y_hat)

Training the model
Training epoch 0/100
Epoch=0 | 33.33%
Epoch=0 | 66.67%
Epoch=0 | 100.00%

Training epoch 1/100
Epoch=1 | 33.33%
Epoch=1 | 66.67%
Epoch=1 | 100.00%

Training epoch 2/100
Epoch=2 | 33.33%
Epoch=2 | 66.67%
Epoch=2 | 100.00%

Training epoch 3/100
Epoch=3 | 33.33%
Epoch=3 | 66.67%
Epoch=3 | 100.00%

Training epoch 4/100
Epoch=4 | 33.33%
Epoch=4 | 66.67%
Epoch=4 | 100.00%

Training epoch 5/100
Epoch=5 | 33.33%
Epoch=5 | 66.67%
Epoch=5 | 100.00%

Training epoch 6/100
Epoch=6 | 33.33%
Epoch=6 | 66.67%
Epoch=6 | 100.00%

Training epoch 7/100
Epoch=7 | 33.33%
Epoch=7 | 66.67%
Epoch=7 | 100.00%

Training epoch 8/100
Epoch=8 | 33.33%
Epoch=8 | 66.67%
Epoch=8 | 100.00%

Training epoch 9/100
Epoch=9 | 33.33%
Epoch=9 | 66.67%
Epoch=9 | 100.00%

Training epoch 10/100
Epoch=10 | 33.33%
Epoch=10 | 66.67%
Epoch=10 | 100.00%

Training epoch 11/100
Epoch=11 | 33.33%
Epoch=11 | 66.67%
Epoch=11 | 100.00%

Training epoch 12/100
Epoch=12 | 33.33%
Epoch=12 | 66.67%
Epoch=12 | 100.00%

Trainin

In [17]:
from sklearn.neural_network import MLPRegressor
clf = MLPRegressor(hidden_layer_sizes=(2,), max_iter=10, solver="sgd",learning_rate_init=0.001, activation="logistic")
for i in range(10):
    clf.fit(X, t.flatten())
print(clf.predict(X_test))

[0.04610885]


C:\Users\andre\anaconda3\envs\pytorch_transformers\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\andre\anaconda3\envs\pytorch_transformers\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\andre\anaconda3\envs\pytorch_transformers\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\andre\anaconda3\envs\pytorch_transformers\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't conv